<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/GPC_(Gemma_Python_Chat).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-assistants-with-gemma:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F64148%2F7669720%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240322%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240322T054640Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5bd6a675fb9b15764ea12c62b83e4658889c025a1c7084704ea3782d1a427eb2156d1b3787f9bb9e9535dc1c46fa838a0825274721cef9f6a4f9feafa5fd902322508c907b9d6167395bb268d97455056e15cf069c731756c8e07948821585c55b7dc28e133e2cc7c6914c5beae7e734a2564cfc8be3b70a3bf1de3e8f2970215a5418d669d001bf4ce3f82919ff9450d17dac4211d2295a7c1eae8c164038cbfea51c4a87094642ec424ecbb9180c18d910b130cfb042ccd36173365e0356ec9ec6b138c710d7c1011f1245b72357520f0f396e85dc60084151334a45ea909e1aa7dd83f1f0b4ad7c7c839bfe45d8e4be3c77face6956676e47dcd225bd9102,pythonquestions:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F262%2F726715%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240322%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240322T054641Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7f774320aff8ecbf8f867a88f4d2edf032b5cb153ea972c0692dc605e1de5c6c3d0e384746b449bb7a30b8b64d5c8a44c708b5ad51048c18a7927994ad28febbd8f7bb55e489b9cc149a8c2504a01b46bff94a6bd91d28dd50a45c81fccde462f61482d63c0ca00e2475a223725a2f239a30cf35ef7a1159aa7260f9cde5d04a837c0466517f1558c5bee8ccd8525cb253e96eaeec26cf67a896607abf486f6315f73bd38da94eac7770b9b64e84269ea1c53640a334e8422ccaf5e699e3747e3baf71213b1595e2facad4e0c9cf8c21d14c663a22a0dd9fe1d3c4e5e595a8b9fb7387176dea61c8ed5eca000fe8dc40822b70b9024c301ca8607c3e022072b8,gemma/transformers/2b/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F6216%2F11384%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240322%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240322T054641Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D74552fbacb0ffa1a43ebb9a21d9afea18f81c6e99a74b95a5b128deb9c8eff747db0a0293e125afa38a765e93469872519ae876029689adb918add244502d1711689d633d6a4dfa9b1c25040a307e9b7cd7ffae2bb6e5e2b0fae903051d21616a11ec08e8fecbe68e920f9b9aa141aaad752995e242cad931cd5b61cba6db8bf80b3d8f963e2973ec6904064e8f5b3cebb33743d3cb7db61d48f95d1f761ac611c99222a9e800f952712c6321292f87973ed8629ae22974d0e1e4ff63627fa705c26bbee05e7893135699760acf26cbed7e49917d7c2ac6425bb657e088db5a16eafd74e76e8ca9eb5b8695e9391072784d7895d66c4d2484844dbb936bdc59c'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# GCP 🤖 Gemma Python Chatbot

![](https://i.ibb.co/8xZNc32/Gemma.png)

The Gemma Python Chatbot 🚀🚀 help you to answer common questions about the 🐍 Python programming language, powereg by [Gemma 7B IT](https://blog.google/technology/developers/gemma-open-models/) updated with the latest

## Requirements

In [ ]:
#base
import os
import json
import csv
import pandas as pd
from typing import *
# model
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
)
# data
from datasets import load_dataset

In [ ]:
model_checkpoint = "/kaggle/input/gemma/transformers/2b/2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, torch_dtype=torch.float16).cuda()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Data

In [ ]:
class pythonQAData:
    """
    Processes data from Questions and Answers CSV files to provide a structured Q&A format.

    Attributes:
        questions_path (str): Path to the Questions CSV file.
        answers_path (str): Path to the Answers CSV file.

    Methods:
        load_data(): Loads the CSV data into DataFrames.
        merge(): Cleans, merges, and formats the question and answer data.
        get_formatted_qa(): Returns a list of formatted question-answer strings.
    """

    def __init__(self, questions_path, answers_path):
        self.questions_path = '../input/pythonquestions/Questions.csv'
        self.answers_path = '../input/pythonquestions/Answers.csv'
        self.answers_path = '../input/pythonquestions/Tags.csv'

    def load_data(self):
        """Loads Questions and Answers data from CSV files."""
        self.df_questions = pd.read_csv(self.questions_path, encoding="ISO-8859-1", usecols=['Id', 'Score', 'Title'])
        self.df_answers = pd.read_csv(self.answers_path, encoding="ISO-8859-1", usecols=['ParentId', 'Score', 'Body'])


    def merge(self):
        """Cleans, merges, and formats the question and answer data."""
        # Filter by score
        self.df_questions = self.df_questions[self.df_questions['Score'] > 0]
        self.df_answers = self.df_answers[self.df_answers['Score'] > 0]

        # Sort and deduplicate answers
        self.df_answers = self.df_answers.sort_values('Score', ascending=False).drop_duplicates(subset=['ParentId'])

        # Merge and rename
        self.qa_data = self.df_questions.merge(self.df_answers, left_on='Id', right_on='ParentId')\
                                        .rename(columns={'Title': 'Question', 'Body': 'Answer'})[['Question', 'Answer', 'Score_x']]

    def get_formatted_qa(self):
        """Returns a list of formatted question-answer strings."""
        data = []
        for index, row in self.qa_data.iterrows():
            data.append(f"Question:\n{row['Question']}\n\nAnswer:\n{row['Answer']}")
        return data


## RAG

## Chatbot

In [ ]:
%%time
# Use the model
input_text = "What is the data cut-off date for the Gemma Google models training dataset?"
input_ids = tokenizer(input_text, return_tensors="pt")
input_ids = {k: v.to("cuda") for k, v in input_ids.items()}  if torch.cuda.is_available() else {k: v.to("cpu")}
outputs = model.generate(**input_ids, max_length=200)
print(tokenizer.decode(outputs[0]))

<bos>What is the data cut-off date for the Gemma Google models training dataset?

The data cut-off date for the Gemma Google models training dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models validation dataset?

The data cut-off date for the Gemma Google models validation dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models test dataset?

The data cut-off date for the Gemma Google models test dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models development dataset?

The data cut-off date for the Gemma Google models development dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models training dataset
CPU times: user 5.48 s, sys: 0 ns, total: 5.48 s
Wall time: 5.47 s


In [ ]:
%%time
# Use the model
input_text = "if 5 t-shirts get dry in 10 hour ho long spend 30 t-shirts to get dry?"
input_ids = tokenizer(input_text, return_tensors="pt")
input_ids = {k: v.to("cuda") for k, v in input_ids.items()}  if torch.cuda.is_available() else {k: v.to("cpu")}
outputs = model.generate(**input_ids, max_length=500)
print(tokenizer.decode(outputs[0]))

## GCP - 0.0.1

## Fine-Tunning

## GCP - 0.1.0